In [ ]:
# default_exp etl

# etl

> Utilerias para extraer y transformar datos sobre delincuencia en la CDMX.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import os
import glob
import itertools
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import timedelta, date, datetime
import seaborn as sns
import requests

## get_carpetas_from_api

In [ ]:
# export
def get_carpetas_from_api(limit=100):
    """Regresa un GeoDataFrame con los primeros `limit` registros de la base abierta.
    
    """
    url = f'https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=48fcb848-220c-4af0-839b-4fd8ac812c0f&limit={limit}'
    r = requests.get(url, allow_redirects=True)
    records = r.json()['result']['records'] 
    records = pd.DataFrame(records)
    records.replace('NA', np.nan, inplace=True)
    records.dropna(subset=['longitud', 'latitud'], how='any', inplace=True)
    records = gpd.GeoDataFrame(records, geometry=gpd.points_from_xy(records.longitud, records.latitud))
    records = records.set_crs(epsg=4326)
    records['fecha_hechos'] = pd.to_datetime(records.fecha_hechos)
    return records

In [ ]:
carpetas = get_carpetas_from_api()
assert type(carpetas) == gpd.GeoDataFrame

## get_historico_carpetas

In [ ]:
# export
def get_historico_carpetas():
    """Regresa un GeoDataFrame con todos los registros de carpetas de investigación.
    
    """
    archivo = "datos/descargas/carpetas_fiscalia.csv"
    url = "https://archivo.datos.cdmx.gob.mx/fiscalia-general-de-justicia/carpetas-de-investigacion-fgj-de-la-ciudad-de-mexico/carpetas_completa_julio_2021.csv"
    r = requests.get(url, allow_redirects=True)
    open(archivo, 'wb').write(r.content)
    records = pd.read_csv(archivo)
    records.replace('NA', np.nan, inplace=True)
    records.dropna(subset=['longitud', 'latitud'], how='any', inplace=True)
    records = gpd.GeoDataFrame(records, geometry=gpd.points_from_xy(records.longitud, records.latitud))
    records = records.set_crs(epsg=4326)
    records['fecha_hechos'] = pd.to_datetime(records.fecha_hechos)
    return records

In [ ]:
carpetas_todas = get_historico_carpetas()
assert type(carpetas_todas) == gpd.GeoDataFrame

/tmp/ipykernel_22689/329436171.py:1: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  carpetas_todas = get_historico_carpetas()


## agrega_ids_espaciales

In [ ]:
# export
def agrega_ids_espaciales(carpetas):
    """Agrega ids de colonias y cuadrantes a la base de carpetas."""
    colonias = gpd.read_file("datos/criminologia_capas.gpkg", layer='colonias').drop(columns='colonia_geom_6362')
    cuadrantes = gpd.read_file("datos/criminologia_capas.gpkg", layer='cuadrantes')
    carpetas = (gpd.tools.sjoin(carpetas, colonias[['colonia_cve', 'colonia_nombre', 'municipio_cvegeo', 'geometry']])
                .drop(columns=['index_right'])
               )
    carpetas = (gpd.tools.sjoin(carpetas, cuadrantes[['cuadrante_id', 'geometry']])
                .drop(columns=['index_right']))
    return carpetas

In [ ]:
carpetas = agrega_ids_espaciales(carpetas)
assert 'colonia_cve' in carpetas.columns
assert 'cuadrante_id' in carpetas.columns
assert 'municipio_cvegeo' in carpetas.columns

## agregar_categorias_de_usuario

In [ ]:
# export
def agregar_categorias_de_usuario(carpetas, archivo_categorias="datos/categorias_incidentes.csv"):
    """Agrega una columna con categorías definidas por el usuario.
    
      Las categorías tienen que venir en un csv con columnas incidente y categoria que
      relacionen las categorías del usuario con la columna delitos de la base de carpetas.
    """
    if 'categoria' in carpetas.columns:
        carpetas = carpetas.drop(columns='categoria')
    if 'incidente' in carpetas.columns:
        carpetas = carpetas.drop(columns='incidente')
    categorias = pd.read_csv(archivo_categorias)
    carpetas = (carpetas
                .merge(categorias, left_on='delito', right_on='incidente', how='left')
                .drop(columns='incidente'))
    return carpetas

In [ ]:
carpetas = agregar_categorias_de_usuario(carpetas)
assert 'categoria' in carpetas.columns

## exporta_datos_vusualizador

In [ ]:
# export
def exporta_datos_vusualizador(carpetas, archivo_resultado):
    """ Escribe en archivo_resultado un csv para consumirse en el visualizador."""
    carpetas['lat'] = carpetas.geometry.y
    carpetas['long'] = carpetas.geometry.x
    carpetas = carpetas[['fecha_hechos', 'delito', 'categoria', 'municipio_cvegeo', 'colonia_cve', 
                         'cuadrante_id', 'categoria', 'lat', 'long']]
    carpetas.to_csv(archivo_resultado)

In [ ]:
exporta_datos_vusualizador(carpetas, "datos/salidas/carpetas.csv")

## serie_de_tiempo_categoria

In [ ]:
# export
def serie_de_tiempo_categoria(carpetas, fecha_inicio, categoria, freq='M'):
    """ Regresa una serie de tiempo con los agregados por `freq` de la `categoria`.
        
        parameters:
        carpetas: incidentes, deben traer la columna categoria
        fecha_inicio: pd.datetime fecha del inicio de la serie
        categoria: nombre de la categoría a agregar (`agregar_categorias_de_usuario`)
        freq: frecuencia de agregación (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)
    """
    carpetas = carpetas.loc[carpetas.fecha_hechos >= fecha_inicio]
    carpetas = carpetas.loc[carpetas.categoria == categoria]
    serie = (carpetas
             .set_index('fecha_hechos')[['categoria']]
             .resample(freq)
             .size()
             .reset_index()
             .rename({0:categoria}, axis=1)
            )
    return serie

In [ ]:
serie = serie_de_tiempo_categoria(carpetas_todas, pd.to_datetime('01/01/2016'), 'Robo a pasajero')
serie

,fecha_hechos,Robo a pasajero
0,2016-01-31,310
1,2016-02-29,265
2,2016-03-31,275
3,2016-04-30,319
4,2016-05-31,256
...,...,...
62,2021-03-31,493
63,2021-04-30,472
64,2021-05-31,500
65,2021-06-30,435


In [ ]:
carpetas

,_id,ao_hechos,mes_hechos,fecha_hechos,ao_inicio,mes_inicio,fecha_inicio,delito,fiscalia,agencia,...,tempo,geometry,colonia_cve,colonia_nombre,municipio_cvegeo,cuadrante_id,categoria,lat,long,dia_hechos
0,1024416,2020,Junio,2020-06-18 11:30:00,2020,Junio,2020-06-18T15:41:11,AMENAZAS,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN BENIT...,BJ-3,...,NaN,POINT (-99.16156 19.37454),932,LETRAN VALLE,09014,013,NaN,19.374543,-99.161563,2020-06-18
1,1024440,2020,Junio,2020-06-18 16:00:00,2020,Junio,2020-06-18T16:25:08,ROBO A NEGOCIO SIN VIOLENCIA POR FARDEROS (TIE...,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN BENIT...,BJ-2,...,NaN,POINT (-99.16448 19.37081),927,STA CRUZ ATOYAC,09014,013,Robo a negocio,19.370805,-99.164477,2020-06-18
2,1024417,2020,Junio,2020-06-13 14:00:00,2020,Junio,2020-06-18T15:42:35,SUSTRACCIÓN DE MENORES,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN IZTAC...,IZC-2,...,NaN,POINT (-99.10572 19.39534),898,TLACOTAL RAMOS MILLAN,09006,025,NaN,19.395336,-99.105717,2020-06-13
3,1024418,2020,Junio,2020-06-18 14:40:00,2020,Junio,2020-06-18T15:42:41,ROBO A NEGOCIO CON VIOLENCIA,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN IZTAP...,IZP-6,...,NaN,POINT (-99.06893 19.31571),389,SAN LORENZO TEZONCO (PBLO),09007,017,Robo a negocio,19.315710,-99.068925,2020-06-18
4,1024419,2020,Mayo,2020-05-30 12:00:00,2020,Junio,2020-06-18T15:45:31,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN IZTAP...,IZP-5,...,NaN,POINT (-99.08480 19.33541),1413,SAN JUAN XALPA I,09007,013,Robo de/en vehículo,19.335411,-99.084796,2020-05-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1024509,2020,Junio,2020-06-03 06:30:00,2020,Junio,2020-06-18T18:19:43,ROBO DE OBJETOS DEL INTERIOR DE UN VEHICULO,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN IZTAP...,IZP-6,...,NaN,POINT (-99.05389 19.38622),418,TEPALCATES I,09007,0119,Robo de/en vehículo,19.386222,-99.053885,2020-06-03
93,1024510,2020,Mayo,2020-05-25 18:00:00,2020,Junio,2020-06-18T18:21:58,CONTRA LA INTIMIDAD SEXUAL,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN XOCHI...,XO-2,...,NaN,POINT (-99.08073 19.25275),1734,XICALHUACAN,09013,046,NaN,19.252751,-99.080732,2020-05-25
94,1024511,2020,Abril,2020-04-22 12:00:00,2020,Junio,2020-06-18T18:21:59,VIOLENCIA FAMILIAR,FISCALÍA DE INVESTIGACIÓN DE DELITOS COMETIDOS...,59,...,NaN,POINT (-99.20923 19.21245),1702,SANTO TOMAS AJUSCO (PBLO),09012,046,NaN,19.212447,-99.209233,2020-04-22
95,1024513,2020,Junio,2020-06-18 14:50:00,2020,Junio,2020-06-18T18:24:18,VIOLACION,FISCALÍA DE INVESTIGACIÓN DE DELITOS SEXUALES,FDS-6,...,NaN,POINT (-99.04474 19.31090),1495,LA ESTACION,09011,031,NaN,19.310901,-99.044741,2020-06-18


In [ ]:
carpetas_todas.shape

(1212892, 27)

In [ ]:
pd.to_datetime('01/01/2016')

Timestamp('2016-01-01 00:00:00')

In [ ]:
c.fecha_hechos.min()

Timestamp('2016-01-01 00:05:00')